In [217]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager
import time
import math
import pandas as pd

### CASA QUERY

In [140]:
# CASA QUERY:

url = 'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
url

'https://www.inmuebles24.com/casas-o-duplex-o-casa-en-condominio-o-casa-uso-de-suelo-o-desarrollo-horizontal-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'

In [197]:
# MULTIPLE PAGES:

number_of_results = soup.find("h1", class_ = "list-result-title").text.split(" ")[0]
number_of_results = int(number_of_results)

total_paging_times = math.ceil(number_of_results/24) + 1
total_paging_times

for x in range(2,total_paging_times):
    url_varias = f'https://www.inmuebles24.com/casas-o-desarrollo-horizontal-o-duplex-o-casa-uso-de-suelo-o-casa-en-condominio-en-venta-en-distrito-federal-pagina-{x}-q-ciudad-de-mexico.html'
    print(url_varias)

https://www.inmuebles24.com/casas-o-desarrollo-horizontal-o-duplex-o-casa-uso-de-suelo-o-casa-en-condominio-en-venta-en-distrito-federal-pagina-2-q-ciudad-de-mexico.html
https://www.inmuebles24.com/casas-o-desarrollo-horizontal-o-duplex-o-casa-uso-de-suelo-o-casa-en-condominio-en-venta-en-distrito-federal-pagina-3-q-ciudad-de-mexico.html
https://www.inmuebles24.com/casas-o-desarrollo-horizontal-o-duplex-o-casa-uso-de-suelo-o-casa-en-condominio-en-venta-en-distrito-federal-pagina-4-q-ciudad-de-mexico.html
https://www.inmuebles24.com/casas-o-desarrollo-horizontal-o-duplex-o-casa-uso-de-suelo-o-casa-en-condominio-en-venta-en-distrito-federal-pagina-5-q-ciudad-de-mexico.html
https://www.inmuebles24.com/casas-o-desarrollo-horizontal-o-duplex-o-casa-uso-de-suelo-o-casa-en-condominio-en-venta-en-distrito-federal-pagina-6-q-ciudad-de-mexico.html
https://www.inmuebles24.com/casas-o-desarrollo-horizontal-o-duplex-o-casa-uso-de-suelo-o-casa-en-condominio-en-venta-en-distrito-federal-pagina-7-q-ci

### DEPARTAMENTO QUERY

In [208]:
# DEPARTAMENTO QUERY:

url = 'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'
url

'https://www.inmuebles24.com/departamento-compartido-o-departamentos-o-desarrollo-vertical-en-venta-en-distrito-federal-q-ciudad-de-mexico.html'

In [209]:
executable_path = {"executable_path": ChromeDriverManager().install()}
browser = Browser("chrome", **executable_path, headless=False)


browser.visit(url)

browser.is_element_not_present_by_id('list-card-container', wait_time=2)

html = browser.html

soup = bs(html, "html.parser")

[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389
[WDM] - Driver [C:\Users\Intel NUC\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [210]:
len(soup.find_all("div", class_ = "postingCard"))

24

In [329]:
sub_soup = publications_list[0]

thumbnail = sub_soup.find_all("img")
thumbnail[0]
# thumbnail_list.append(thumbnail)

# for x in range(len(thumbnail)):
#     print(thumbnail[0]['data-src'].strip())

<img alt="" class="flickity-lazyloaded" height="266" src="https://img10.naventcdn.com/avisos/18/00/52/91/37/25/360x266/75401701.jpg" width="360"/>

In [332]:
publications_list = soup.find_all("div", class_ = "postingCard")

permalink_list = []
titles_list = []
price_list = []
bedrooms_number_list = []
full_bathrooms_list = []
thumbnail_list = []
announcement_id_list = []

type_of_operation=[]
property_type=[]
currency=[]

state=[]
city=[]
colonia=[]
address_street = []


# registration_date = []
real_state_agency = []


properties_feature_list = []


count_number = 0

for x in range(len(publications_list)):
    
    count_number += 1
    
#   PROPERTY_ID
    property_id = publications_list[x]["data-id"]
    announcement_id_list.append(property_id)
    
    
#     HYPERLINK    
    hyperlink = f'https://www.inmuebles24.com{publications_list[x]["data-to-posting"]}'
    permalink_list.append(hyperlink)

#     ------SUB_SOUP-----
    
    sub_soup = publications_list[x]
    
#     PROPERTY_TITLE
    ad_title = sub_soup.find_all("h2", class_ = "postingCardTitle")[0].get_text(strip=True)
    titles_list.append(ad_title)
    
#     PROPERTY PRICE
    property_price = sub_soup.find_all("span", class_ = "firstPrice")[0].get_text(strip=True).replace('MN ', '').replace(',', '')
    price_list.append(property_price)
    
# #     print(f'id: {property_id} , title: {ad_title} , {hyperlink} , price: {property_price}')
    
#     THUMBNAIL:

    try:
        thumbnail = sub_soup.find_all("img")[0]['data-src'].strip()
        thumbnail_list.append(thumbnail)
    except:
        thumbnail = sub_soup.find_all("img")[0]['src'].strip()
        thumbnail_list.append(thumbnail)
    
#     FEATURES LIST:
    
    inside_list = []
    
    features_list = sub_soup.find_all("ul", class_ = "postingCardMainFeatures")[0].find_all("li")
    
    for j in range(len(features_list)):
        feature = features_list[j].get_text(strip = True).replace("\n", " ").replace("\t", "")
        inside_list.append(feature)
        
#         BEDROOM NUMBER:

        if 'rec.' in feature:
            string = feature[0:2]
            bedrooms_number_list.append(string)
            
#         BATHROOMS NUMBER:

        if 'baños' in feature or 'baño' in feature:
            string = feature[0:2]
            full_bathrooms_list.append(string)
    
    if len(full_bathrooms_list) != count_number:
        full_bathrooms_list.append("not_available")
    
    properties_feature_list.append(inside_list)


In [335]:
pd.DataFrame({
    'announcement_id_list': announcement_id_list,
    'titles_list': titles_list,
    'price_list': price_list,
    'bedrooms_number_list': bedrooms_number_list,
    'full_bathrooms_list': full_bathrooms_list,
    'permalink_list': permalink_list,
    'thumbnail_list': thumbnail_list,
})

,announcement_id_list,titles_list,price_list,bedrooms_number_list,full_bathrooms_list,permalink_list,thumbnail_list
0,52913725,Bosque 6060,11312978,3,not_available,https://www.inmuebles24.com/propiedades/bosque...,https://img10.naventcdn.com/avisos/18/00/52/91...
1,54456938,Sky Tower Lindavista,3210000,2,not_available,https://www.inmuebles24.com/propiedades/sky-to...,https://img10.naventcdn.com/avisos/18/00/54/45...
2,55534814,"Último Departamento, Departamento en Venta Tla...",7950000,3,not_available,https://www.inmuebles24.com/propiedades/ultimo...,https://img10.naventcdn.com/avisos/18/00/55/53...
3,60679141,Chicle 160,2052000,2,not_available,https://www.inmuebles24.com/propiedades/chicle...,https://img10.naventcdn.com/avisos/18/00/60/67...
4,56515561,Up Santa Fe,3701095,1,not_available,https://www.inmuebles24.com/propiedades/up-san...,https://img10.naventcdn.com/avisos/18/00/56/51...
5,60405953,Departamento en Lisboa Ciudad de México,5500000,2,2,https://www.inmuebles24.com/propiedades/depart...,https://img10.naventcdn.com/avisos/18/00/60/40...
6,60859941,Departamento en Pennsylvania Ciudad de México,4480000,4,2,https://www.inmuebles24.com/propiedades/depart...,https://img10.naventcdn.com/avisos/18/00/60/85...
7,60732379,Departamento en Progreso Ciudad de México,3650000,3,2,https://www.inmuebles24.com/propiedades/depart...,https://img10.naventcdn.com/avisos/18/00/60/73...
8,60716573,Departamento en Tanana Ciudad de México,3650000,3,2,https://www.inmuebles24.com/propiedades/depart...,https://img10.naventcdn.com/avisos/18/00/60/71...
9,60438499,"Venta Departamento en Tlalpan, Ciudad de México",2650000,2,2,https://www.inmuebles24.com/propiedades/venta-...,https://img10.naventcdn.com/avisos/18/00/60/43...


In [181]:
features_list = sub_soup.find_all("ul", class_ = "postingCardMainFeatures")[0].find_all("li")

additional_list = []

for x in range(len(features_list)):
    test = features_list[x].get_text(strip = True).replace("\n", " ").replace("\t", "")
    additional_list.append(test)
#     print(test)

additional_list

['6 deptos.', '3 a 4 rec.', '189 a 208   m²']

## Scrape of each webpage

In [221]:
# THIS SHOULD BE OBTAINED FROM EACH PAGES' WEBSITE:

square_meters_list = []
unit_list = []

item_condition = []

if 'm²' in feature:
    square_meters_list.append(feature.replace("m²", ""))
    unit_list.append('m²') 

In [116]:
import time

image_details_coordinates_list = []
latitude_list = []
longitude_list = []

count_number = 0

for x in range(3):
    
    count_number += 1

    url = hyperlink_list[x]

    browser.visit(url)

    time.sleep(1)

    html = browser.html

    soup = bs(html, "html.parser")
    map_image = soup.find('div', class_ = 'article-map')

    image_details = map_image.img['src'].split("&")

    print(hyperlink_list[x])
    
    for x in range(len(image_details)):
        if 'markers' in image_details[x]:
            image_details_coordinates = image_details[x].replace('markers=', '').split(",")
            image_details_coordinates_list.append(image_details_coordinates)
            
            latitude = image_details_coordinates[0]
            longitude = image_details_coordinates[1]            
            latitude_list.append(latitude)
            longitude_list.append(longitude)
                        
    if len(image_details_coordinates_list) != count_number:
        for x in range(len(image_details)):
            if 'center' in image_details[x]:
                image_details_coordinates = image_details[x].replace('center=', '').split(",")
                image_details_coordinates_list.append(image_details_coordinates)

                latitude = image_details_coordinates[0].split("?")[1]
                longitude = image_details_coordinates[1]            
                latitude_list.append(latitude)
                longitude_list.append(longitude)

    print(latitude, longitude)

https://www.inmuebles24.com/propiedades/bosque-6060-52913725.html
19.331155400000000 -99.256939199999980
https://www.inmuebles24.com/propiedades/sky-tower-lindavista-54456938.html
19.481846100000000 -99.129833500000020
https://www.inmuebles24.com/propiedades/ultimo-departamento-departamento-en-venta-tlalpan-55534814.html
19.279131400000000 -99.140570900000000


In [261]:
pd.DataFrame({
    "bedrooms_number" : bedrooms_number_list
})

,bedrooms_number
0,3
1,2
2,3
3,2
4,1
5,2
6,4
7,3
8,3
9,2


In [126]:
browser.quit()